## 1. Import packages

In [1]:
#!pip install tweepy

In [2]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [3]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [4]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [6]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(7500)

### 3.2. Retreive the tweets

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

7214it [14:56,  8.05it/s]


In [8]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 7214


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 7214/7214 [00:38<00:00, 188.00it/s]


In [10]:
tweets_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Hassaan Ahmed Siddiqui,"Islamabad, Pakistan","Deutsch studentin, Arzt zu sein 👨‍⚕️",2016-03-23 06:49:23,29,16,94,False,2020-08-06 16:40:36,"Lockdown Ends, but it doesn't mean that we all...",None,Twitter for iPhone,False
0,Jonah Currie,"Surrey , BC, Canada","healer, teacher, alchemist, nature lover, ligh...",2013-01-31 01:55:58,2547,5000,2337,False,2020-08-06 16:40:35,"""Our New Normal"" Is Code for the New World Ord...","[COVID19, coronavirus, EndTheNightmare]",Twitter for iPad,False
0,State Representative Natalie M. Blais,"Sunderland, MA",Official twitter account for State Representat...,2017-08-30 19:54:05,764,803,1359,False,2020-08-06 16:40:34,Just as we in the state legislature have made ...,[COVID19],TweetDeck,False
0,Hal Newnan,,School Board Trustee and Sierra Club Activist,2012-05-06 11:52:50,543,1712,19960,False,2020-08-06 16:40:34,This could be a school district; this wouldn’t...,[UnitedShore],Twitter for iPhone,False
0,hairandyoga,Dallas,"Hairstylist at Anderson,Cox, and Company at St...",2013-03-28 12:37:46,95,367,6189,False,2020-08-06 16:40:33,I'm standing with @DoorDash to support my loca...,None,Twitter for iPad,False


## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (74436, 13)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 7214 past tweets: 74436 all tweets: 81650


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (81650, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)